Predicting high-value purchases in retail
Build a binary-calssification model to predict:
HIGH value purchase:
    1 --> Purchase >= 10000
    2 --> Purchase < 10000
Steps to do so:
    1. Data Preparation 
        cleaning null values and redundant values, encode categorical variables (Gender,Age,City_Category,etc),CReate a binary target column:high_value_purchase, drop unnecessary columns, scale features
    2.Build Baseline Model
        Logistic provide confusion matrix and Keras(activation func: sigmoid)(optimizer:binary_crossentropy and adam)(use ReLU in 2 hidden layers)
    3.Compare both the models

In [2]:
import pandas as pd 
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

In [3]:
df1 = pd.read_csv("user_demographics.csv")
df2 = pd.read_csv("User_product_purchase_details_p2.csv")

In [4]:
df1.head()

,User_ID,Gender,Age,Occupation
0,1000001,F,0-17,10
1,1000002,M,55+,16
2,1000003,M,26-35,15
3,1000004,M,46-50,7
4,1000005,M,26-35,20


In [5]:
df2.head()

,User_ID,Product_ID,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,C,4+,0,8,NaN,NaN,7969


In [7]:
merge_df = pd.merge(df1,df2, on='User_ID',how='left')
merge_df.head()

,User_ID,Gender,Age,Occupation,Product_ID,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,F,0-17,10,P00069042,A,2,0,3,NaN,NaN,8370
1,1000001,F,0-17,10,P00248942,A,2,0,1,6.0,14.0,15200
2,1000001,F,0-17,10,P00087842,A,2,0,12,NaN,NaN,1422
3,1000001,F,0-17,10,P00085442,A,2,0,12,14.0,NaN,1057
4,1000001,F,0-17,10,P00085942,A,2,0,2,4.0,8.0,12842


In [15]:
merge_df.to_csv("mergedata.csv",index= False)

In [11]:
merge_df['High_Value_Purchase'] = (merge_df["Purchase"] >= 10000).astype(int)
merge_df.head()

,User_ID,Gender,Age,Occupation,Product_ID,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,High_Value_Purchase
0,1000001,F,0-17,10,P00069042,A,2,0,3,NaN,NaN,8370,0
1,1000001,F,0-17,10,P00248942,A,2,0,1,6.0,14.0,15200,1
2,1000001,F,0-17,10,P00087842,A,2,0,12,NaN,NaN,1422,0
3,1000001,F,0-17,10,P00085442,A,2,0,12,14.0,NaN,1057,0
4,1000001,F,0-17,10,P00085942,A,2,0,2,4.0,8.0,12842,1


In [13]:
from sklearn.model_selection import train_test_split
merge_df = merge_df.drop(["Product_ID"], axis = 1)
merge_df.head()

,User_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,High_Value_Purchase
0,1000001,F,0-17,10,A,2,0,3,NaN,NaN,8370,0
1,1000001,F,0-17,10,A,2,0,1,6.0,14.0,15200,1
2,1000001,F,0-17,10,A,2,0,12,NaN,NaN,1422,0
3,1000001,F,0-17,10,A,2,0,12,14.0,NaN,1057,0
4,1000001,F,0-17,10,A,2,0,2,4.0,8.0,12842,1


In [19]:
# Data Cleaning and Preprocessing
merge_df = merge_df.fillna(0)

In [20]:
#  checking if purchase contains any null values
merge_df.isnull().sum()
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Gender                      550068 non-null  object 
 2   Age                         550068 non-null  object 
 3   Occupation                  550068 non-null  int64  
 4   City_Category               550068 non-null  object 
 5   Stay_In_Current_City_Years  550068 non-null  object 
 6   Marital_Status              550068 non-null  int64  
 7   Product_Category_1          550068 non-null  int64  
 8   Product_Category_2          550068 non-null  float64
 9   Product_Category_3          550068 non-null  float64
 10  Purchase                    550068 non-null  int64  
 11  High_Value_Purchase         550068 non-null  int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 50.4+ MB


In [28]:
# Converting categorical columns to numerical using one-hot encoding
# ensure columns exist before one-hot encoding to avoid KeyError
categorical_cols = ['Gender','Age','Occupation','City_Category','Stay_In_Current_City_Years','Marital_Status']
cols_to_encode = [c for c in categorical_cols if c in merge_df.columns]

if cols_to_encode:
    merge_df = pd.get_dummies(merge_df, columns=cols_to_encode, drop_first=True)
else:
    print("No categorical columns found to encode. Available columns:", list(merge_df.columns))
# merge_df.head()

In [29]:
merge_df.head()

,User_ID,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,High_Value_Purchase,Gender_F,Gender_M,Age_0-17,Age_18-25,...,Occupation_20,City_Category_A,City_Category_B,City_Category_C,Stay_In_Current_City_Years_0,Stay_In_Current_City_Years_1,Stay_In_Current_City_Years_2,Stay_In_Current_City_Years_3,Stay_In_Current_City_Years_4+,Marital_Status_1
0,1000001,3,0.0,0.0,8370,0,True,False,True,False,...,False,True,False,False,False,False,True,False,False,False
1,1000001,1,6.0,14.0,15200,1,True,False,True,False,...,False,True,False,False,False,False,True,False,False,False
2,1000001,12,0.0,0.0,1422,0,True,False,True,False,...,False,True,False,False,False,False,True,False,False,False
3,1000001,12,14.0,0.0,1057,0,True,False,True,False,...,False,True,False,False,False,False,True,False,False,False
4,1000001,2,4.0,8.0,12842,1,True,False,True,False,...,False,True,False,False,False,False,True,False,False,False


In [30]:
# Data Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = merge_df.drop(['High_Value_Purchase','Purchase'],axis=1)
X = scaler.fit_transform(X)
y = merge_df["High_Value_Purchase"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [31]:
#  Model Building 
#  Build the Logistic Regression model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(y_pred)

[0 0 1 ... 0 0 0]


In [42]:
# Evaluate the Logistic Regression model
from sklearn.metrics import classification_report, confusion_matrix,precision_score, recall_score, f1_score
cm = confusion_matrix(y_test, y_pred)
print("Evaluate the Logistic Regression model")
print("Confusion Matrix",cm)
print("Precision:",precision_score(y_test,y_pred))
print("Recall:",recall_score(y_test,y_pred))
print("F1 Score:",f1_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))

Evaluate the Logistic Regression model
Confusion Matrix [[64625  7380]
 [18096 19913]]
Precision: 0.7296009965925329
Recall: 0.5239022336814965
F1 Score: 0.6098741233040336
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.90      0.84     72005
           1       0.73      0.52      0.61     38009

    accuracy                           0.77    110014
   macro avg       0.76      0.71      0.72    110014
weighted avg       0.76      0.77      0.76    110014



In [37]:
# Model BUilding using simple MLP 
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['precision', 'recall'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
y_pred_mlp = (model.predict(X_test) > 0.5).astype("int32")

Epoch 1/10


c:\Users\User\AIML Training\Unsupervised Learning\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11002/11002 ━━━━━━━━━━━━━━━━━━━━ 12s 969us/step - loss: 0.4060 - precision: 0.7689 - recall: 0.7027 - val_loss: 0.3305 - val_precision: 0.7725 - val_recall: 0.8258
Epoch 2/10
11002/11002 ━━━━━━━━━━━━━━━━━━━━ 11s 983us/step - loss: 0.3073 - precision: 0.7857 - recall: 0.8635 - val_loss: 0.2991 - val_precision: 0.7775 - val_recall: 0.8930
Epoch 3/10
11002/11002 ━━━━━━━━━━━━━━━━━━━━ 11s 958us/step - loss: 0.2902 - precision: 0.7892 - recall: 0.8941 - val_loss: 0.2850 - val_precision: 0.7883 - val_recall: 0.9143
Epoch 4/10
11002/11002 ━━━━━━━━━━━━━━━━━━━━ 10s 921us/step - loss: 0.2814 - precision: 0.7900 - recall: 0.9073 - val_loss: 0.2821 - val_precision: 0.7932 - val_recall: 0.8929
Epoch 5/10
11002/11002 ━━━━━━━━━━━━━━━━━━━━ 10s 935us/step - loss: 0.2766 - precision: 0.7900 - recall: 0.9151 - val_loss: 0.2768 - val_precision: 0.7874 - val_recall: 0.9236
Epoch 6/10
11002/11002 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 0.2735 - precision: 0.7905 - recall: 0.9199 - val_loss: 0.2743 - val_pr

In [43]:
# Evaluate the MLP model
cm_mlp = confusion_matrix(y_test, y_pred_mlp)
print("Evaluate the MLP model")
print("Confusion Matrix MLP",cm_mlp)
print("Precision MLP:",precision_score(y_test,y_pred_mlp))
print("Recall MLP:",recall_score(y_test,y_pred_mlp))
print("F1 Score MLP:",f1_score(y_test,y_pred_mlp))
print("Classification Report MLP:\n",classification_report(y_test,y_pred_mlp))

Evaluate the MLP model
Confusion Matrix MLP [[63386  8619]
 [ 3286 34723]]
Precision MLP: 0.8011397720455908
Recall MLP: 0.9135467915493699
F1 Score MLP: 0.85365883640029
Classification Report MLP:
               precision    recall  f1-score   support

           0       0.95      0.88      0.91     72005
           1       0.80      0.91      0.85     38009

    accuracy                           0.89    110014
   macro avg       0.88      0.90      0.88    110014
weighted avg       0.90      0.89      0.89    110014



In [44]:
# Compare the performance of both models and choose the best one for deployment.
eval = {
    "mlp": {
        "confusion_matrix": cm_mlp,
        "precision": precision_score(y_test, y_pred_mlp),
        "recall": recall_score(y_test, y_pred_mlp),
        "f1_score": f1_score(y_test, y_pred_mlp)
    },
    "logistic_regression": {
        "confusion_matrix": cm,
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1_score": f1_score(y_test, y_pred)
    }
}
print("Model Evaluation Comparison:")
print(eval)

Model Evaluation Comparison:
{'mlp': {'confusion_matrix': array([[63386,  8619],
       [ 3286, 34723]]), 'precision': 0.8011397720455908, 'recall': 0.9135467915493699, 'f1_score': 0.85365883640029}, 'logistic_regression': {'confusion_matrix': array([[64625,  7380],
       [18096, 19913]]), 'precision': 0.7296009965925329, 'recall': 0.5239022336814965, 'f1_score': 0.6098741233040336}}
